In [ ]:
from pool import AMMPool, AMMPoolHistory, CPMM

In [ ]:
import pandas as pd 
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
import sysconfig
print(sysconfig.get_python_version())

**Define Pools to study and initialize**

In [ ]:
# Define token0=USDC and token1=WETH
token0 = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
token1 = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'

**Get Uni trades stream**  

In [ ]:
from datastreams.datastream import Streamer

query_size = 10000

# instantiate Streamer object. 
# Note - unlike the cow queries, univ3 does not require multiple streamer instantations because the swaps field path is reset each iteration. 
# If the Cow queries were updated to use the same method, we could use the same streamer object for all queries.
univ3_ds = Streamer('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')

# get a query field path from the query dictionary which is automatically populated in the Streamer object
uni_swaps_fp = univ3_ds.queryDict.get('swaps')

# define query path and parameters 
uni_swaps_qp = uni_swaps_fp(
        first=query_size, 
        orderBy='timestamp',
        orderDirection='asc',
        where = {'timestamp_gt':1643673600, 'pool': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'} 
        )

# run query
uni_swaps_df = univ3_ds.runQuery(uni_swaps_qp)

In [ ]:
uni_swaps_df.dtypes

In [ ]:
uni_swaps_df.head(5)

In [ ]:
# Find the first and last value in the timestamp column
first_timestamp_uni = uni_swaps_df['swaps_timestamp'].min()
last_timestamp_uni = uni_swaps_df['swaps_timestamp'].max()
print(first_timestamp_uni, last_timestamp_uni)

**get chainlink pricing to value pools**

In [ ]:
# load streamer class
chain_ds = Streamer('https://api.thegraph.com/subgraphs/name/openpredict/chainlink-prices-subgraph')
chain_price_feed = "ETH/USD"
chain_dfs_list = []

In [ ]:
# get a query field path from the query dictionary which is automatically populated in the Streamer object
chain_fp = chain_ds.queryDict.get('prices')

# add parameters to the settlements_qp.
chain_qp = chain_fp(
    first= 10000,
    orderBy='timestamp',
    orderDirection='asc',
    where = {'timestamp_gt':1643673600, 'assetPair': chain_price_feed}
    )

# run query
chain_df = chain_ds.runQuery(chain_qp)

In [ ]:
# drop prices_id, endpoint. 
chain_df = chain_df.drop(['prices_id', 'endpoint'], axis=1)
# divide prices_price by 1e8 to get the price in USD
chain_df['prices_price'] = chain_df['prices_price'] / 10 ** 8

In [ ]:
chain_df.shape

In [ ]:
chain_df.head(5)

In [ ]:
print(chain_df.columns) 

In [ ]:
# Find the first and last value in the timestamp column
first_timestamp = chain_df['prices_timestamp'].min()
last_timestamp = chain_df['prices_timestamp'].max()
print(first_timestamp, last_timestamp)

In [ ]:
timestamps = pd.DataFrame({'prices_timestamp': range(first_timestamp, last_timestamp + 1)})

In [ ]:
timestamps.shape

In [ ]:
chain_df = pd.merge(timestamps, chain_df, on='prices_timestamp', how='left')

In [ ]:
chain_df.head(5)

In [ ]:
# Forward-fill missing values
chain_df.fillna(method='ffill', inplace=True)
print(chain_df.shape)
chain_df

**get initialization price**

In [ ]:
first_timestamp_uni

In [ ]:
# chainlink at first_timestamp of uni trades:
initialization_df = chain_df[chain_df["prices_timestamp"] == first_timestamp]
initilizatioin_price = initialization_df.iloc[0,2]
initilizatioin_price

In [ ]:
USDC_reserves_USD = 10000000000


token0_start = USDC_reserves_USD*10**6
print(token0_start)
token1_start = int(USDC_reserves_USD/initilizatioin_price)*(10**9)
print(token1_start)

**1.Simple AMM. SWAP ALL** 

In [ ]:
# Create an instance of AMMPoolHistory 
feedlot1 = AMMPoolHistory(token0_start, token1_start)
# sanity check: should be USDC/WETH price
print(feedlot1.reserves[0] / feedlot1.reserves[1] * (10**3) )

In [ ]:
# Instantiate the CPMM object with the AMMPool object
cpmm = CPMM(pool=feedlot1)

In [ ]:
# Iterate through the rows of the dataframe
for index, row in uni_swaps_df.iterrows():
    # Determine if the swap is a buy or a sell
    token_in = row['swaps_tokenIn_id']
    token_out = row['swaps_tokenOut_id']
    is_buy = token_in == token0 and token_out == token1
    amt = row['swaps_amountIn']
    
    # Extract the amountIn, amountOut, and timestamp
    if is_buy:
        token = 0
    else:
        token = 1 
        amt /= 1000000000 # adjust because we shaved off 9 decimals
    ts = row['swaps_timestamp']
    
    # Run the swap function on the AMM pool
    cpmm.market_order_sell(amt,token,ts)

In [ ]:
results1 = pd.DataFrame(feedlot1.history)

In [ ]:
prices=(results1["reserve0"] / results1["reserve1"]) * 1000
prices

In [ ]:
ax0 =results1["reserve0"].plot()
ax1 = ax0.twinx()
ax1 =results1["reserve1"].plot()

In [ ]:
merged1_df = pd.merge(results1, chain_df, left_on='ts', right_on='prices_timestamp')
merged1_df.rename(columns={'prices_price': 'chainlink_price'}, inplace=True)

In [ ]:
merged1_df

In [ ]:
merged1_df['value'] = (merged1_df['reserve1']*merged1_df['chainlink_price']/(10**18)) + merged1_df['reserve0']/(10**18)

In [ ]:
merged1_df

In [ ]:
merged1_df.isna().sum()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(merged1_df['ts'], merged1_df['value'])
ax.set_title('Value vs. Timestamp')
ax.set_xlabel('Timestamp')
ax.set_ylabel('Value')
#fig.savefig('value_vs_timestamp.png', dpi=300)


In [ ]:
fig, ax = plt.subplots()
ax.plot(merged1_df['ts'], merged1_df['reserve0'])
ax.set_title('CPMM reserves0')
ax.set_xlabel('Timestamp')
ax.set_ylabel('Value')
#fig.savefig('value_vs_timestamp.png', dpi=300)

In [ ]:
fig, ax = plt.subplots()
ax.plot(merged1_df['ts'], merged1_df['reserve1'])
ax.set_title('CPMM reserves1')
ax.set_xlabel('Timestamp')
ax.set_ylabel('Value')
#fig.savefig('value_vs_timestamp.png', dpi=300)

In [ ]:
import matplotlib.pyplot as plt

# Assume that df is your dataframe with three columns: time, col1, and col2

# Create a new figure and axis
fig, ax = plt.subplots()

# Plot col1 on the first y-axis
ax.plot(merged1_df["ts"], merged1_df["reserve0"], label="reserve0")

# Create a second y-axis on the right-hand side
ax2 = ax.twinx()

# Plot col2 on the second y-axis
ax2.plot(merged1_df["ts"], merged1_df["reserve1"], color="orange", label="col2")

# Add legends and axis labels
ax.set_xlabel("Timestamp")
ax.set_ylabel("reserve0")
ax2.set_ylabel("reserve1")
ax.legend(loc="upper left")
ax2.legend(loc="upper right")

# Show the plot
plt.show()


**OraclePool AMM**

**get cow trades for oracle pricing** 

In [ ]:
cow_ds = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')

# get COW trades 

trades_fp = cow_ds.queryDict.get('trades')

# trades query path that gets token a -> token b trades
trades_qp = trades_fp(
    first=10000,
    orderBy='timestamp',
    orderDirection='asc',
    where={'timestamp_gt':1643673600, 'sellToken':'0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
           'buyToken':'0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
    }
)

# run query
trades_df = cow_ds.runQuery(trades_qp)

In [ ]:
trades_df.shape

In [ ]:
trades_df.head(5)

In [ ]:
# Set query size large enough to retrieve all tokens in the graph
query_size = 10000

token_fp = cow_ds.queryDict.get('tokens')

# add parameters to the settlements_qp.
token_qp = token_fp(
    first=query_size,
)

# run query
token_df = cow_ds.runQuery(token_qp)


In [ ]:
token_df.shape

In [ ]:
# Merge1 on sell token address.  x = sell token 

merged_df = pd.merge(trades_df, token_df, left_on='trades_sellToken_id', right_on='tokens_address')

merged_df.shape

In [ ]:
# Merge2 on buy token address.  y = buy token 
cow_df = pd.merge(merged_df, token_df, left_on='trades_buyToken_id', right_on='tokens_address')

cow_df.shape

In [ ]:
cow_df.dtypes

In [ ]:
# calculate buy and sell amounts from the correct decimal 
cow_df['sell_amount'] = cow_df.apply(lambda x: x['trades_sellAmount'] / (10**x['tokens_decimals_x']), axis=1)
cow_df['buy_amount'] = cow_df.apply(lambda x: x['trades_buyAmount'] / (10**x['tokens_decimals_y']), axis=1)

In [ ]:
# calculate COW price defined sell amount / buy amount
cow_df['cow_price'] = cow_df['sell_amount'] / cow_df['buy_amount']

In [ ]:
cow_df

In [ ]:
# Find the first and last value in the timestamp column
first_timestamp_cow = chain_df['prices_timestamp'].min()
last_timestamp_cow = chain_df['prices_timestamp'].max()

In [ ]:
first_timestamp_cow

In [ ]:
last_timestamp_cow

In [ ]:
oracle_dict = [{"ts": row["trades_timestamp"], "p": row["cow_price"]} for _, row in cow_df.iterrows()]

In [ ]:
oracle_dict

**Oracle pool simulation**

In [ ]:
from pool import CPMM, AMMPool, SwapRejectedError, OraclePool

In [ ]:
# Create an instance of AMMPoolHistory 
feedlot2 = AMMPoolHistory(token0_start, token1_start)

In [ ]:
feedlot3 = OraclePool(feedlot2, oracle_dict)

In [ ]:
 # Iterate through the rows of the dataframe
for index, row in uni_swaps_df.iterrows():
    # Determine if the swap is a buy or a sell
    token_in = row['swaps_tokenIn_id']
    token_out = row['swaps_tokenOut_id']
    is_buy = token_in == token0 and token_out == token1
    amt = row['swaps_amountIn']
    
    # Extract the amountIn, amountOut, and timestamp
    if is_buy:
        token = 0
    else:
        token = 1 
    ts = row['swaps_timestamp']
    
    print('token', token)
    print('amt', amt)
    print('amt',amt/10**18)
    print('token_in', token_in)
    print('ts', ts)
    
    # Run the swap function on the AMM pool
    feedlot3.market_order_sell(amt,token,ts)

In [ ]:
results2 = pd.DataFrame(feedlot2.history)

In [ ]:
results2.shape

In [ ]:
results2.head(5)

In [ ]:
merged2_df = pd.merge(results2, chain_df, left_on='ts', right_on='prices_timestamp')
merged2_df.rename(columns={'prices_price': 'chainlink_price'}, inplace=True)

In [ ]:
merged2_df['value'] = merged2_df['reserve1']*merged2_df['chainlink_price']/10**18 + merged2_df['reserve0']/10**18

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(merged2_df['ts'], merged2_df['value'])
ax.set_title('Value vs. Timestamp')
ax.set_xlabel('Timestamp')
ax.set_ylabel('Value')
#fig.savefig('value_vs_timestamp.png', dpi=300)

In [ ]:
# Assume that df is your dataframe with three columns: time, col1, and col2

# Create a new figure and axis
fig, ax = plt.subplots()

# Plot col1 on the first y-axis
ax.plot(merged2_df["ts"], merged2_df["reserve0"], label="reserve0")

# Create a second y-axis on the right-hand side
ax2 = ax.twinx()

# Plot col2 on the second y-axis
ax2.plot(merged2_df["ts"], merged2_df["reserve1"], color="orange", label="reserves1")

# Add legends and axis labels
ax.set_xlabel("Timestamp")
ax.set_ylabel("reserve0")
ax2.set_ylabel("reserve1")
ax.legend(loc="upper left")
ax2.legend(loc="upper right")

# Show the plot
plt.show()